In [2]:
using CSV
using DataFrames
using Z3
using JuMP
using HiGHS

In [ ]:
# model = Model(HiGHS.Optimizer)
# @variable(model, x[1:length(sv)], Bin);
# @constraint(model, c[k=1:length(is)], sum([x[i] for i = is[k]]) >= 1);
# @objective(model, Min, sum(x));
# optimize!(model)
# println("Optimal value: ", objective_value(model))

In [50]:
model = JuMP.Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [51]:
@variable(model, x[1:5,1:10,1:3], Bin)

5×10×3 Array{VariableRef, 3}:
[:, :, 1] =
 x[1,1,1]  x[1,2,1]  x[1,3,1]  x[1,4,1]  …  x[1,8,1]  x[1,9,1]  x[1,10,1]
 x[2,1,1]  x[2,2,1]  x[2,3,1]  x[2,4,1]     x[2,8,1]  x[2,9,1]  x[2,10,1]
 x[3,1,1]  x[3,2,1]  x[3,3,1]  x[3,4,1]     x[3,8,1]  x[3,9,1]  x[3,10,1]
 x[4,1,1]  x[4,2,1]  x[4,3,1]  x[4,4,1]     x[4,8,1]  x[4,9,1]  x[4,10,1]
 x[5,1,1]  x[5,2,1]  x[5,3,1]  x[5,4,1]     x[5,8,1]  x[5,9,1]  x[5,10,1]

[:, :, 2] =
 x[1,1,2]  x[1,2,2]  x[1,3,2]  x[1,4,2]  …  x[1,8,2]  x[1,9,2]  x[1,10,2]
 x[2,1,2]  x[2,2,2]  x[2,3,2]  x[2,4,2]     x[2,8,2]  x[2,9,2]  x[2,10,2]
 x[3,1,2]  x[3,2,2]  x[3,3,2]  x[3,4,2]     x[3,8,2]  x[3,9,2]  x[3,10,2]
 x[4,1,2]  x[4,2,2]  x[4,3,2]  x[4,4,2]     x[4,8,2]  x[4,9,2]  x[4,10,2]
 x[5,1,2]  x[5,2,2]  x[5,3,2]  x[5,4,2]     x[5,8,2]  x[5,9,2]  x[5,10,2]

[:, :, 3] =
 x[1,1,3]  x[1,2,3]  x[1,3,3]  x[1,4,3]  …  x[1,8,3]  x[1,9,3]  x[1,10,3]
 x[2,1,3]  x[2,2,3]  x[2,3,3]  x[2,4,3]     x[2,8,3]  x[2,9,3]  x[2,10,3]
 x[3,1,3]  x[3,2,3]  x[3,3,3]  x[3,4,3]     

In [54]:
maximum.([sum(x[i,:,:]) for i = 1:5])

5-element Vector{AffExpr}:
 x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,7,1] + x[1,8,1] + x[1,9,1] + x[1,10,1] + x[1,1,2] + x[1,2,2] + x[1,3,2] + x[1,4,2] + x[1,5,2] + x[1,6,2] + x[1,7,2] + x[1,8,2] + x[1,9,2] + x[1,10,2] + x[1,1,3] + x[1,2,3] + x[1,3,3] + x[1,4,3] + x[1,5,3] + x[1,6,3] + x[1,7,3] + x[1,8,3] + x[1,9,3] + x[1,10,3]
 x[2,1,1] + x[2,2,1] + x[2,3,1] + x[2,4,1] + x[2,5,1] + x[2,6,1] + x[2,7,1] + x[2,8,1] + x[2,9,1] + x[2,10,1] + x[2,1,2] + x[2,2,2] + x[2,3,2] + x[2,4,2] + x[2,5,2] + x[2,6,2] + x[2,7,2] + x[2,8,2] + x[2,9,2] + x[2,10,2] + x[2,1,3] + x[2,2,3] + x[2,3,3] + x[2,4,3] + x[2,5,3] + x[2,6,3] + x[2,7,3] + x[2,8,3] + x[2,9,3] + x[2,10,3]
 x[3,1,1] + x[3,2,1] + x[3,3,1] + x[3,4,1] + x[3,5,1] + x[3,6,1] + x[3,7,1] + x[3,8,1] + x[3,9,1] + x[3,10,1] + x[3,1,2] + x[3,2,2] + x[3,3,2] + x[3,4,2] + x[3,5,2] + x[3,6,2] + x[3,7,2] + x[3,8,2] + x[3,9,2] + x[3,10,2] + x[3,1,3] + x[3,2,3] + x[3,3,3] + x[3,4,3] + x[3,5,3] + x[3,6,3] + x[3,7,3] + x[3,8,3] 

In [6]:
x[1]

x[1]

In [60]:
nslot = 17 # 時間帯割り当て
narea = 2 # エリア数
requirenum = 1 # 必要人数
totaltime = 10 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
csvfile = "testdata.csv"

model = JuMP.Model(HiGHS.Optimizer)

df = CSV.read(csvfile, DataFrame)
nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end"] # 名前取得
println(nms)

y = zeros(Int, length(nms), nslot)
for (i,(k,nm)) = enumerate(nms)
    for t = 1:nslot
        if ismissing(df[t,k])
            y[i,t] = 1 # 無記入は「可能」
        else
            y[i,t] = df[t,k]
        end
    end
end

@variable(model, x[1:length(nms),1:nslot,1:narea], Bin)

for i = size(y)[1]
    for t = 1:nslot
        @constraint(model, sum(x[i,t,:]) <= y[i,t]) # 可能時間帯に高々一つの割り当て
    end
end

for t = 1:nslot
    for k = 1:narea
        @constraint(model, sum(x[:,t,k]) >= requirenum) # 各時間帯で必要人数確保
    end
end

for i = size(y)[1]
    @constraint(model, sum(x[i,:,:]) <= totaltime) # 労働時間制限
end

# op = []
# for i = size(y)[1]
#     for k = 1:narea
#         tmp = x[i,1,k]
#         for t = 2:nslot
#             tmp += (x[i,t,k] - x[i,t-1,k]) * (x[i,t,k] - x[i,t-1,k])
#         end
#         push!(op, tmp)
#         # @constraint(model, tmp <= totalswitch)
#     end
# end

# for (nm,xs) = area
#     for k = 1:narea
#         tmp = Z3.int_val(ctx, 0)
#         for t = 1:nslot
#             if t == 1
#                 tmp += xs[t][k]
#             else
#                 tmp += abs(xs[t][k] - xs[t-1][k])
#             end
#         end
#         push!(cons, tmp <= totalswitch) # 場所替え数制約
#     end
# end

@objective(model, Min, sum(op));


[(3, "山田"), (4, "田中"), (5, "鈴木"), (6, "佐藤")]


In [61]:
optimize!(model)
println("Optimal value: ", objective_value(model))

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
ERROR:   Cannot solve MIQP problems with HiGHS
Optimal value: 0.0


In [62]:
Int.(JuMP.value.(x))

4×17×2 Array{Int64, 3}:
[:, :, 1] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

[:, :, 2] =
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [ ]:
"""
nslot = 17 # 時間帯割り当て
narea = 2 # エリア数
requirenum = 1 # 必要人数
totaltime = 15 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
"""

function maketable(csvfile ;nslot = 17, narea = 2, requirenum = 1, totaltime = 15, totalswitch = 2)
    model = Model(HiGHS.Optimizer)

    df = CSV.read(csvfile, DataFrame)
    nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end"] # 名前取得
    println(nms)

    y = zeros(Int, 1:length(nms), 1:nslot)
    for (i,(k,nm)) = enumerate(nms)
        for t = 1:nslot
            if ismissing(df[t,k])
                y[i,t] = 1
            else
                y[i,t] = df[t,k]
            end
        end
    end

    @variable(model, x[1:length(nms),1:nslot,1:narea], Bin)

    for (i,nm) = nms
        tmparea = []
        for t = 1:nslot
            @constraint(model, c[], sum(x[i,t,:]) <= df[t,i]);
            # x = Z3.int_const(ctx, "var$(i)_$(t)")
            # f = [Z3.int_const(ctx, "area$(k)_$(i-2)_$(t)") for k = 1:narea] # 2 はオフセットフォーマットに合わせる必要あり
            push!(tmparea, f)
            for v = f
                push!(cons, v >= 0) # 割り当て変数非負条件
            end
            push!(cons, sum(f) <= x) # 可能時間帯に高々一つの割り当て
            if ismissing(df[t,i])
                push!(cons, x == 1) # 無記入は「可能」
            else
                push!(cons, x == Int(df[t,i])) # 可能な時間帯の指定
            end
        end
        area[nm] = tmparea
    end

    for t = 1:nslot
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for (_,xs) = area
                tmp += xs[t][k]
            end
            push!(cons, tmp == requirenum) # 各時間帯で必要人数確保
        end
    end

    for (nm,xs) = area
        tmp = Z3.int_val(ctx, 0)
        for x = xs
            tmp += sum(x)
        end
        push!(cons, tmp <= totaltime) # 労働時間制限
    end

    for (nm,xs) = area
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for t = 1:nslot
                if t == 1
                    tmp += xs[t][k]
                else
                    tmp += abs(xs[t][k] - xs[t-1][k])
                end
            end
            push!(cons, tmp <= totalswitch) # 場所替え数制約
        end
    end

    solver = Z3.Solver(ctx)
    for x = cons
        add(solver, x)
    end

    res = Z3.check(solver)
    if res == Z3.sat
        m = Z3.get_model(solver)
        result = Dict(["$k"=>Z3.get_numeral_int(v) for (k,v) = Z3.consts(m)])
    else
        error("No solution")
    end

    table = Dict()
    for (i,nm) = enumerate(keys(area))
        tmp = []
        for t = 1:nslot
            u = [result["area$(k)_$(i)_$(t)"] for k = 1:narea]
            res = findfirst(x->x==1, u)
            if isnothing(res)
                push!(tmp, 0)
            else
                push!(tmp, res)
            end
        end
        table[nm] = tmp
    end
    DataFrame(table)
end

In [2]:
"""
nslot = 17 # 時間帯割り当て
narea = 2 # エリア数
requirenum = 1 # 必要人数
totaltime = 15 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
"""

function maketable(csvfile ;nslot = 17, narea = 2, requirenum = 1, totaltime = 15, totalswitch = 2)
    ctx = Z3.Context()

    cons = []
    area = Dict()

    df = CSV.read(csvfile, DataFrame)

    nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end"] # 名前取得
    println(nms)

    for (i,nm) = nms
        tmparea = []
        for t = 1:nslot
            x = Z3.int_const(ctx, "var$(i)_$(t)")
            f = [Z3.int_const(ctx, "area$(k)_$(i-2)_$(t)") for k = 1:narea] # 2 はオフセットフォーマットに合わせる必要あり
            push!(tmparea, f)
            for v = f
                push!(cons, v >= 0) # 割り当て変数非負条件
            end
            push!(cons, sum(f) <= x) # 可能時間帯に高々一つの割り当て
            if ismissing(df[t,i])
                push!(cons, x == 1) # 無記入は「可能」
            else
                push!(cons, x == Int(df[t,i])) # 可能な時間帯の指定
            end
        end
        area[nm] = tmparea
    end

    for t = 1:nslot
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for (_,xs) = area
                tmp += xs[t][k]
            end
            push!(cons, tmp == requirenum) # 各時間帯で必要人数確保
        end
    end

    for (nm,xs) = area
        tmp = Z3.int_val(ctx, 0)
        for x = xs
            tmp += sum(x)
        end
        push!(cons, tmp <= totaltime) # 労働時間制限
    end

    for (nm,xs) = area
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for t = 1:nslot
                if t == 1
                    tmp += xs[t][k]
                else
                    tmp += abs(xs[t][k] - xs[t-1][k])
                end
            end
            push!(cons, tmp <= totalswitch) # 場所替え数制約
        end
    end

    solver = Z3.Solver(ctx)
    for x = cons
        add(solver, x)
    end

    res = Z3.check(solver)
    if res == Z3.sat
        m = Z3.get_model(solver)
        result = Dict(["$k"=>Z3.get_numeral_int(v) for (k,v) = Z3.consts(m)])
    else
        error("No solution")
    end

    table = Dict()
    for (i,nm) = enumerate(keys(area))
        tmp = []
        for t = 1:nslot
            u = [result["area$(k)_$(i)_$(t)"] for k = 1:narea]
            res = findfirst(x->x==1, u)
            if isnothing(res)
                push!(tmp, 0)
            else
                push!(tmp, res)
            end
        end
        table[nm] = tmp
    end
    DataFrame(table)
end

maketable (generic function with 1 method)

In [7]:
maketable("testdata.csv", nslot = 17, narea = 2, requirenum = 1, totaltime = 13, totalswitch = 2)

[(3, "山田"), (4, "田中"), (5, "鈴木"), (6, "佐藤")]


Row,佐藤,山田,田中,鈴木
,Any,Any,Any,Any
1,0,1,2,0
2,2,1,0,0
3,1,2,0,0
4,1,2,0,0
5,1,0,0,2
6,0,0,1,2
7,0,0,1,2
8,0,0,1,2
9,0,0,1,2


In [20]:
df

UndefVarError: UndefVarError: df not defined

In [21]:
df = CSV.read("testdata.csv", DataFrame)

Row,start,end,山田,田中,鈴木,佐藤
,Time,Time,Missing,Int64,Int64,Int64
1,08:00:00,08:10:00,missing,0,1,1
2,08:10:00,08:20:00,missing,1,1,1
3,08:20:00,08:30:00,missing,1,1,1
4,08:30:00,08:40:00,missing,1,1,1
5,08:40:00,08:50:00,missing,1,1,1
6,08:50:00,09:00:00,missing,0,1,0
7,09:00:00,09:10:00,missing,0,1,0
8,09:10:00,09:20:00,missing,0,1,0
9,09:20:00,09:30:00,missing,0,1,0


In [24]:
df["山田"]

ArgumentError: ArgumentError: syntax df[column] is not supported use df[!, column] instead